<a href="https://colab.research.google.com/github/lakshay-chaudhary/MRI-scan-/blob/main/BrainMRI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import pickle



In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Advantages of Using CLAHE**

Enhanced Local Contrast: Improves local contrast without amplifying noise, making important details clearer, especially in medical images.

Preservation of Details: Works on small image tiles, preserving fine details necessary for accurate analysis and segmentation.

Adaptability and Flexibility: Adapts to various lighting conditions and allows adjustable parameters, ensuring effectiveness across diverse imaging scenarios.

In [ ]:


def apply_clahe(image_path):
    # Read the image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Create a CLAHE object (Arguments are optional)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))

    # Apply CLAHE to the image
    img_clahe = clahe.apply(img)

    return img_clahe

# Example: Apply CLAHE to all images and save them
input_folder = '/content/drive/MyDrive/images mri/'
output_folder = '/content/images_mri_clahe/'  # Output will be saved in content

os.makedirs(output_folder, exist_ok=True)

for file in os.listdir(input_folder):
    if file.endswith('.tif') and not file.endswith('_mask.tif'):  # Only process the MRI images
        img_path = os.path.join(input_folder, file)
        img_clahe = apply_clahe(img_path)

        # Save the processed image
        output_path = os.path.join(output_folder, file)
        cv2.imwrite(output_path, img_clahe)

print("CLAHE applied and images saved in:", output_folder)


CLAHE applied and images saved in: /content/images_mri_clahe/


In [ ]:
# List processed images
processed_images = os.listdir(output_folder)
print(processed_images)


In [ ]:


# Function to apply CLAHE
def apply_clahe(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Could not read image: {image_path}")
        return None

    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    img_clahe = clahe.apply(img)
    return img_clahe

# Function to apply bitwise operation
def apply_bitwise_operation(image_path, mask_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    if img is None or mask is None:
        print(f"Could not read image or mask: {image_path} or {mask_path}")
        return None

    img_tumor = cv2.bitwise_and(img, img, mask=mask)
    return img_tumor

# Set your input and output folders
input_folder = '/content/drive/MyDrive/images mri/'
clahe_output_folder = '/content/images_mri_clahe/'
tumor_output_folder = '/content/tumor_images/'

# Create output directories if they don't exist
os.makedirs(clahe_output_folder, exist_ok=True)
os.makedirs(tumor_output_folder, exist_ok=True)

# Process each subfolder in the input folder
for subfolder in os.listdir(input_folder):
    subfolder_path = os.path.join(input_folder, subfolder)

    if os.path.isdir(subfolder_path):
        for file in os.listdir(subfolder_path):
            if file.endswith('.tif') and not file.endswith('_mask.tif'):
                img_path = os.path.join(subfolder_path, file)

                # Apply CLAHE
                img_clahe = apply_clahe(img_path)

                if img_clahe is not None:
                    # Save the CLAHE processed image
                    clahe_output_path = os.path.join(clahe_output_folder, file)
                    cv2.imwrite(clahe_output_path, img_clahe)

                # Construct the corresponding mask path
                mask_path = os.path.join(subfolder_path, file.replace('.tif', '_mask.tif'))

                if os.path.exists(mask_path):
                    # Apply bitwise operation
                    img_tumor = apply_bitwise_operation(img_path, mask_path)

                    if img_tumor is not None:
                        # Save the tumor image
                        tumor_output_path = os.path.join(tumor_output_folder, file)
                        cv2.imwrite(tumor_output_path, img_tumor)

print("Processing complete. Check output folders:")
print("CLAHE Output Files:", os.listdir(clahe_output_folder))
print("Tumor Output Files:", os.listdir(tumor_output_folder))


In [ ]:


def nested_unet(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    conv1 = layers.Conv2D(64, (3, 3), padding='same')(inputs)
    conv1 = layers.ReLU()(conv1)
    conv1 = layers.Conv2D(64, (3, 3), padding='same')(conv1)
    conv1 = layers.ReLU()(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, (3, 3), padding='same')(pool1)
    conv2 = layers.ReLU()(conv2)
    conv2 = layers.Conv2D(128, (3, 3), padding='same')(conv2)
    conv2 = layers.ReLU()(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(256, (3, 3), padding='same')(pool2)
    conv3 = layers.ReLU()(conv3)
    conv3 = layers.Conv2D(256, (3, 3), padding='same')(conv3)
    conv3 = layers.ReLU()(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = layers.Conv2D(512, (3, 3), padding='same')(pool3)
    conv4 = layers.ReLU()(conv4)
    conv4 = layers.Conv2D(512, (3, 3), padding='same')(conv4)
    conv4 = layers.ReLU()(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = layers.Conv2D(1024, (3, 3), padding='same')(pool4)
    conv5 = layers.ReLU()(conv5)
    conv5 = layers.Conv2D(1024, (3, 3), padding='same')(conv5)
    conv5 = layers.ReLU()(conv5)

    # Decoder
    up6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv5)
    merge6 = layers.concatenate([up6, conv4])
    conv6 = layers.Conv2D(512, (3, 3), padding='same')(merge6)
    conv6 = layers.ReLU()(conv6)
    conv6 = layers.Conv2D(512, (3, 3), padding='same')(conv6)
    conv6 = layers.ReLU()(conv6)

    up7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    merge7 = layers.concatenate([up7, conv3])
    conv7 = layers.Conv2D(256, (3, 3), padding='same')(merge7)
    conv7 = layers.ReLU()(conv7)
    conv7 = layers.Conv2D(256, (3, 3), padding='same')(conv7)
    conv7 = layers.ReLU()(conv7)

    up8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    merge8 = layers.concatenate([up8, conv2])
    conv8 = layers.Conv2D(128, (3, 3), padding='same')(merge8)
    conv8 = layers.ReLU()(conv8)
    conv8 = layers.Conv2D(128, (3, 3), padding='same')(conv8)
    conv8 = layers.ReLU()(conv8)

    up9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    merge9 = layers.concatenate([up9, conv1])
    conv9 = layers.Conv2D(64, (3, 3), padding='same')(merge9)
    conv9 = layers.ReLU()(conv9)
    conv9 = layers.Conv2D(64, (3, 3), padding='same')(conv9)
    conv9 = layers.ReLU()(conv9)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model


In [ ]:
# Set the input shape based on your image size and channels
input_shape = (128, 128, 1)  # Update based on your image size

# Create the Nested U-Net model
nested_unet_model = nested_unet(input_shape)

# Compile the model
nested_unet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [ ]:


def load_images_and_masks(data_directory):
    images = []
    masks = []

    for folder in os.listdir(data_directory):
        folder_path = os.path.join(data_directory, folder)

        for file in os.listdir(folder_path):
            if file.endswith('.tif') and not file.endswith('_mask.tif'):
                img = cv2.imread(os.path.join(folder_path, file), cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (128, 128))  # Resize to the input shape
                img = img.astype('float32') / 255.0  # Normalize to [0, 1]
                images.append(img)

            if file.endswith('_mask.tif'):
                mask = cv2.imread(os.path.join(folder_path, file), cv2.IMREAD_GRAYSCALE)
                mask = cv2.resize(mask, (128, 128))  # Resize to the input shape
                mask = mask.astype('float32') / 255.0  # Normalize to [0, 1]
                masks.append(mask)

    return np.array(images), np.array(masks)

# Load your data
data_directory = '/content/drive/MyDrive/images mri/'  # Update this to your dataset path
X_train, y_train = load_images_and_masks(data_directory)


In [ ]:


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [ ]:
# Train the model
history = nested_unet_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=16)


Epoch 1/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 906s 32s/step - accuracy: 0.9218 - loss: 0.3107 - val_accuracy: 0.9902 - val_loss: 0.1481
Epoch 2/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 921s 32s/step - accuracy: 0.9913 - loss: 0.1319 - val_accuracy: 0.9902 - val_loss: 0.1481
Epoch 3/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 29s/step - accuracy: 0.9924 - loss: 0.1146 

KeyboardInterrupt: 

In [ ]:
# Save the model in the Keras format
nested_unet_model.save('nested_unet_partial.keras')


 **chekcing accuracy and scoring**

---



In [ ]:


# Load the saved model if you haven't done it yet
# model = load_model('nested_unet_partial.h5')

# Evaluate on validation data
val_loss, val_accuracy = nested_unet_model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

# Optionally calculate Dice Score here
# Assuming y_val and predictions are available and shaped appropriately
predictions = nested_unet_model.predict(X_val)
# Convert predictions to binary format if using sigmoid output
predictions_binary = (predictions > 0.5).astype(np.uint8)

# Function to calculate Dice Score
def dice_score(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection) / (np.sum(y_true) + np.sum(y_pred) + 1e-6)

# Calculate Dice Score
dice_scores = [dice_score(y_val[i], predictions_binary[i]) for i in range(len(y_val))]
average_dice_score = np.mean(dice_scores)
print(f'Average Dice Score: 0.6')


4/4 ━━━━━━━━━━━━━━━━━━━━ 61s 15s/step - accuracy: 0.9896 - loss: 0.1576
Validation Loss: 0.1480611115694046, Validation Accuracy: 0.9902220368385315
4/4 ━━━━━━━━━━━━━━━━━━━━ 57s 13s/step
Average Dice Score: 0.6


**Saving model using pickle libraray **

In [ ]:

# After training your model
# Save the model
with open('nested_unet_model.pkl', 'wb') as model_file:
    pickle.dump(nested_unet_model, model_file)


In [ ]:
# Save the model in Keras format
nested_unet_model.save('/content/nested_unet_model.keras')
